In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
!unzip "UCI HAR Dataset.zip"

# PRE-PROCESSING

In [ ]:
df_train = pd.read_csv("UCI HAR Dataset/train/X_train.txt", delim_whitespace = True,names=list(range(0,561)))

df_train['Subject'] = pd.read_csv("UCI HAR Dataset/train/subject_train.txt", header = None, squeeze = True)

df_train["activity"] = pd.read_csv("UCI HAR Dataset/train/y_train.txt", header = None, squeeze = True)

In [ ]:
df_test= pd.read_csv("UCI HAR Dataset/test/X_test.txt", delim_whitespace = True,names=list(range(0,561)))

df_test['Subject'] = pd.read_csv("UCI HAR Dataset/test/subject_test.txt", header = None, squeeze = True)

df_test["activity"] = pd.read_csv("UCI HAR Dataset/test/y_test.txt", header = None, squeeze = True)

In [ ]:
df_train.to_csv('training.csv',index_label=False)
df_test.to_csv('testing.csv',index_label=False)

In [ ]:
X_train=df_train.iloc[:,:-1]
y_train=df_train.iloc[:,-1]

In [ ]:
X_test=df_test.iloc[:,:-1]
y_test=df_test.iloc[:,-1]

In [ ]:
sensors=['body_acc_x_train','body_acc_y_train','body_acc_z_train','body_gyro_x_train','body_gyro_y_train','body_gyro_z_train','total_acc_x_train','total_acc_y_train','total_acc_z_train']

In [ ]:
training_samples=[]
testing_samples=[]

In [ ]:
for i in sensors:
  training_samples.append(np.array(pd.read_csv("UCI HAR Dataset/train/Inertial Signals/"+i+".txt",delim_whitespace=True,header=None)))

In [ ]:
sensors=['body_acc_x_test','body_acc_y_test','body_acc_z_test','body_gyro_x_test','body_gyro_y_test','body_gyro_z_test','total_acc_x_test','total_acc_y_test','total_acc_z_test']

In [ ]:
for i in sensors:
  testing_samples.append(np.array(pd.read_csv("UCI HAR Dataset/test/Inertial Signals/"+i+".txt",delim_whitespace=True,header=None)))

In [ ]:
training_samples=np.transpose(training_samples,(1,2,0))

In [ ]:
target_train=pd.read_csv("UCI HAR Dataset/train/y_train.txt",header=None)
target_train -= 1

In [ ]:
target_train=np.array(target_train)

In [ ]:
testing_samples=np.transpose(testing_samples,(1,2,0))

In [ ]:
target_test=pd.read_csv("UCI HAR Dataset/test/y_test.txt",header=None)

In [ ]:
target_test -= 1

In [ ]:
target_test=np.array(target_test)

In [ ]:
from keras.utils.np_utils import to_categorical
categorical_training=to_categorical(target_train)
categorical_testing=to_categorical(target_test)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
model=Sequential()
model.add(LSTM(128,return_sequences=True,input_shape=(128,9)))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(6,activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(training_samples,categorical_training,epochs=8,validation_split=0.05)

Epoch 1/8
219/219 [==============================] - 46s 207ms/step - loss: 0.1141 - accuracy: 0.9517 - val_loss: 0.0920 - val_accuracy: 0.9239
Epoch 2/8
219/219 [==============================] - 45s 205ms/step - loss: 0.1184 - accuracy: 0.9513 - val_loss: 0.1027 - val_accuracy: 0.9212
Epoch 3/8
219/219 [==============================] - 45s 204ms/step - loss: 0.1172 - accuracy: 0.9558 - val_loss: 0.1338 - val_accuracy: 0.9158
Epoch 4/8
219/219 [==============================] - 45s 205ms/step - loss: 0.1047 - accuracy: 0.9543 - val_loss: 0.1105 - val_accuracy: 0.9212
Epoch 5/8
219/219 [==============================] - 45s 204ms/step - loss: 0.1262 - accuracy: 0.9489 - val_loss: 0.2106 - val_accuracy: 0.9158
Epoch 6/8
219/219 [==============================] - 45s 204ms/step - loss: 0.1184 - accuracy: 0.9539 - val_loss: 0.1946 - val_accuracy: 0.8967
Epoch 7/8
219/219 [==============================] - 45s 205ms/step - loss: 0.1039 - accuracy: 0.9563 - val_loss: 0.0919 - val_accuracy:

In [ ]:
lstm_predicted=model.predict(testing_samples)

In [ ]:
final_prediction=np.argmax(lstm_predicted,axis=1)

In [ ]:
import sklearn.metrics as acc
acc.accuracy_score(target_test,final_prediction)

0.9280624363759755

# RANDOM FOREST & DECISION TREE

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_model=RandomForestClassifier(criterion='gini',min_samples_split=2,n_estimators=100)
random_model.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


RandomForestClassifier()

In [ ]:
random_model_predicted=random_model.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
import sklearn.metrics as acc
acc.accuracy_score(y_test,random_model_predicted)

0.9300984051577876

In [ ]:
from sklearn.tree import DecisionTreeClassifier
decision_model=DecisionTreeClassifier(criterion='gini',splitter='best',min_samples_split=4)
decision_model.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


DecisionTreeClassifier(min_samples_split=4)

In [ ]:
decision_model_predicted=decision_model.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
acc.accuracy_score(y_test,decision_model_predicted)

0.8625721072276892

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.model_selection import GridSearchCV

In [ ]:
knn_model = knn(n_jobs=-1)
distance_criteria = ['euclidean','manhattan'] 
clusters = np.arange(1,16)
parameters  = dict(metric=distance_criteria, n_neighbors=clusters)
final_knn = GridSearchCV(knn_model, parameters, cv=10, scoring='accuracy', refit=True)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
final_knn.fit(X_train,y_train)

In [ ]:
print("The Best Hyper Parameters: ",final_knn.best_params_)

The Best Hyper Parameters:  {'metric': 'manhattan', 'n_neighbors': 10}


In [ ]:
knn_predicted = final_knn.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
acc.accuracy_score(y_test,knn_predicted)

0.9107567017305734

# SVC

In [ ]:
from sklearn.svm import SVC  
svc_model = SVC(kernel='linear', random_state=0)  
svc_model.fit(X_train, y_train)  

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


SVC(kernel='linear', random_state=0)

In [ ]:
svc_predicted= svc_model.predict(X_test) 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
acc.accuracy_score(y_test,svc_predicted)

0.9633525619273838

# LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression(max_iter=5000)
logistic_model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


LogisticRegression(max_iter=5000)

In [ ]:
logistic_predicted=logistic_model.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
acc.accuracy_score(y_test,logistic_predicted)

0.9596199524940617

# SVM

In [ ]:
from sklearn import svm
svm_classifier = svm.SVC(kernel='rbf')
svm_classifier.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


SVC()

In [ ]:
svm_predicted=svm_classifier.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
acc.accuracy_score(y_test,svm_predicted)

0.9307770614183916

# XGBOOST

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
xgboost_model=XGBClassifier()
xgboost_model.fit(X_train,y_train)

XGBClassifier(objective='multi:softprob')

In [ ]:
xgboost_predicted=xgboost_model.predict(X_test)

In [ ]:
acc.accuracy_score(y_test,xgboost_predicted)


0.9389209365456397